https://adventofcode.com/2022/day/8

In [1]:
f = open("input.txt", "r")

data = []

while ! eof(f)
	s = readline(f)
	
	# push each line in data array
	push!(data, s)
end

close(f)

In [2]:
# creates a 2d array of tree heights from the data
function generateGrid(data)
	grid = []
	for row in data[1:end]
		trees = split(row,"")
		trees = map((tree) -> parse(Int, tree), trees)
		push!(grid, trees)
	end
	return grid
end

# checks if a tree given by [y, x] coordinates is visible from the outside
function isVisible(treeCoordinate,grid)
	# checks if a tree is visible from a line of trees from a given direction
	function isVisibleFromLine(tree, line)
		# loops through the line of trees
		for nextTree in line
			# checks if a tree in the line is taller/same than target tree
			if nextTree >= tree
				return false
			end
		end

		return true
	end

	# checks if the tree is visible from the right
	function isVisibleFromRight(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		# gets all the tree right of the target tree in an array
		rightTrees = grid[y][x+1:end]

		return isVisibleFromLine(tree, rightTrees)
	end

	function isVisibleFromLeft(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		leftTrees = grid[y][begin:x-1]

		return isVisibleFromLine(tree, leftTrees)
	end

	function isVisibleFromBottom(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		bottomTrees = []

		for row in grid[y+1:end]
			bottomTree = row[x]
			push!(bottomTrees,bottomTree)
		end

		return isVisibleFromLine(tree, bottomTrees)
	end

	function isVisibleFromTop(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		topTrees = []

		for row in grid[begin:y-1]
			topTree = row[x]
			push!(topTrees,topTree)
		end

		return isVisibleFromLine(tree, topTrees)
	end
	
	# if the tree is visible from any direction it's visible
	isVisibleFromOutside = isVisibleFromRight(treeCoordinate) ||
		isVisibleFromLeft(treeCoordinate) ||
		isVisibleFromBottom(treeCoordinate) ||
		isVisibleFromTop(treeCoordinate)

	return isVisibleFromOutside
end

# gets the total number of visible trees
function getVisibleTreeCount(data)
	grid = generateGrid(data)

	visibleTreeCount = 0
	# loops through the 2d array and count if each tree is visible
	for y = 1:length(grid), x = 1:length(grid[1])
		treeCoordinate = [y, x]
		if isVisible(treeCoordinate, grid)
			visibleTreeCount += 1
		end
	end

	return visibleTreeCount
end

getVisibleTreeCount(data)

1662

In [3]:
# calculates the scenic score
function getScenicScore(treeCoordinate,grid)
	# counts every tree from of the line until it meets a tree taller/same than the target
	function getScoreFromLine(tree, line)
		score = 0
		for nextTree in line
			score += 1
			if nextTree >= tree
				break
			end
		end

		return score
	end
	
	# calculates the scenic score from the right of the target tree
	function scenicScoreFromRight(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		rightTrees = grid[y][x+1:end]

		score = getScoreFromLine(tree,rightTrees)

		return score
	end

	function scenicScoreFromLeft(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		leftTrees = grid[y][begin:x-1]

		# the order of trees looking from the left of the target is reverses from the top down grid view
		leftTrees = reverse(leftTrees)

		score = getScoreFromLine(tree,leftTrees)

		return score
	end

	function scenicScoreFromBottom(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		bottomTrees = []

		for row in grid[y+1:end]
			bottomTree = row[x]
			push!(bottomTrees,bottomTree)
		end

		score = getScoreFromLine(tree,bottomTrees)

		return score
	end

	function scenicScoreFromTop(coordinate)
		y, x = coordinate
		tree = grid[y][x]

		topTrees = []

		for row in grid[begin:y-1]
			topTree = row[x]
			push!(topTrees,topTree)
		end

		# the order of trees looking from the above of the target is reverses from the top down grid view
		topTrees = reverse(topTrees)

		score = getScoreFromLine(tree,topTrees)

		return score
	end
	
	# multiplies the scores together
	scenicScore = scenicScoreFromRight(treeCoordinate) *
		scenicScoreFromLeft(treeCoordinate) *
		scenicScoreFromBottom(treeCoordinate) *
		scenicScoreFromTop(treeCoordinate)

	return scenicScore
end

function getHighestScenicScore(data)
	grid = generateGrid(data)

	highestScenicScore = 0
	
	for y = 1:length(grid), x = 1:length(grid[1])
		treeCoordinate = [y, x]
		scenicScore = getScenicScore(treeCoordinate, grid)

		if scenicScore > highestScenicScore
			highestScenicScore = scenicScore
		end
	end

	return highestScenicScore
end

getHighestScenicScore(data)

537600

using Julia's matrix features

In [4]:
function generateMatrix(data)
	array = []
	
	for row in data
		trees = split(row,"")
		trees = map((tree) -> parse(Int, tree), trees)
		append!(array, trees)
	end

	height = length(data)
	width = length(data[1])

	return reshape(array, (height, width))
end

# checks if a tree is visible from a line of trees from a given direction
function isVisibleFromLine(tree, line)
	# checks if a tree in the line is taller/same than target tree
	return findfirst((lineTree)-> lineTree >= tree, line) == nothing
end

# checks if a tree given by [x, y] coordinates is visible from the outside
function isVisible(coordinate, matrix)
	x, y = coordinate
	tree = matrix[x, y]

	rightLine = matrix[x+1:end, y]
	leftLine = matrix[x-1:-1:1, y]
	bottomLine = matrix[x, y+1:end]
	topLine = matrix[x, y-1:-1:1]

	# if the tree is visible from any direction it's visible
	isVisibleFromOutside = 
		isVisibleFromLine(tree, rightLine) ||
		isVisibleFromLine(tree, leftLine) ||
		isVisibleFromLine(tree, bottomLine) ||
		isVisibleFromLine(tree, topLine)

	return isVisibleFromOutside
end

# gets the total number of visible trees
function getVisibleTreeCount(data)
	matrix=generateMatrix(data)
	height,width = size(matrix)

	visibleTreeCount = 0

	# loops through the 2d array and count if each tree is visible
	for x = 1:width, y = 1:height 
		coordinate = (x, y)

		if isVisible(coordinate, matrix)
			visibleTreeCount += 1
		end
	end

	return visibleTreeCount
end

getVisibleTreeCount(data)

1662

In [5]:
# counts every tree from of the line until it meets a tree taller/same than the target
function getScoreFromLine(tree, line)
	score = findfirst((lineTree)-> lineTree >= tree, line)
	return score != nothing ? score : length(line)
end

# calculates the scenic score
function getScenicScore(coordinate, matrix)
	x, y = coordinate
	tree = matrix[x, y]

	rightLine = matrix[x+1:end, y]
	leftLine = matrix[x-1:-1:1, y]
	bottomLine = matrix[x, y+1:end]
	topLine = matrix[x, y-1:-1:1]
	
	# multiplies the scores together
	scenicScore = 
		getScoreFromLine(tree, rightLine) *
		getScoreFromLine(tree, leftLine) *
		getScoreFromLine(tree, bottomLine) *
		getScoreFromLine(tree, topLine)

	return scenicScore
end

function getHighestScenicScore(data)
	matrix=generateMatrix(data)
	height,width = size(matrix)

	highestScenicScore = 0

	for x = 1:width, y = 1:height 
		coordinate = (x, y)
		scenicScore = getScenicScore(coordinate, matrix)
		highestScenicScore = max(scenicScore, highestScenicScore)
	end

	return highestScenicScore
end

getHighestScenicScore(data)

537600